# Двунаправленная LSTM

Импортируем необходимые зависимости

In [15]:
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.preprocessing import sequence
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

Загрузим текст для обучения из txt файла

In [16]:
with open('texts/american_psycho.txt', 'r', encoding='utf-8') as file:
    text = file.read()

text = re.sub(r'[^a-zA-Z\s]', '', text)

In [18]:
text

'April Fools\n   Abandon all hope ye who enter here is scrawled in blood red lettering on the side of the Chemical Bank near the corner of Eleventh and First and is in print large enough to be seen from the backseat of the cab as it lurches forward in the traffic leaving Wall Street and just as Timothy Price notices the words a bus pulls up the advertisement forLes Misrableson its side blocking his view but Price who is with Pierce Pierce and twentysix doesnt seem to care because he tells the driver he will give him five dollars to turn up the radio Be My Baby on WYNN and the driver black not American does so\n   Im resourceful Price is saying Im creative Im young unscrupulous highly motivated highly skilled In essence what Im saying is that society cannotafford to lose me Im anasset Price calms down continues to stare out the cabs dirty window probably at the word FEAR sprayed in red graffiti on the side of a McDonalds on Fourth and Seventh I mean the fact remains that no one gives a 

Создадим токенизатор и словарь слов

In [19]:
tokens = word_tokenize(text)

word_dict = {}
for word in tokens:
    if word not in word_dict:
        word_dict[word] = len(word_dict) + 1

reverse_word_dict = {v: k for k, v in word_dict.items()}

sequences = [word_dict[word] for word in tokens]

Создадим обучающие примеры

In [20]:
X = []
y = []
sequence_length = 50

for i in range(len(sequences) - sequence_length):
    X.append(sequences[i:i + sequence_length])
    y.append(sequences[i + sequence_length])

X = np.array(X)
y = to_categorical(y, num_classes=len(word_dict) + 1)

Создадим и обучим модель

In [21]:
embedding_dim = 50
hidden_units = 128

model = Sequential()
model.add(Embedding(input_dim=len(word_dict) + 1, output_dim=embedding_dim, input_length=sequence_length))
model.add(Bidirectional(LSTM(hidden_units)))
model.add(Dense(len(word_dict) + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
1097/1097 [==============================] - 512s 464ms/step - loss: 7.3474 - accuracy: 0.0463
Epoch 2/20
1097/1097 [==============================] - 532s 485ms/step - loss: 6.7270 - accuracy: 0.0783
Epoch 3/20
1097/1097 [==============================] - 563s 513ms/step - loss: 6.2927 - accuracy: 0.1014
Epoch 4/20
1097/1097 [==============================] - 541s 494ms/step - loss: 5.9455 - accuracy: 0.1183
Epoch 5/20
1097/1097 [==============================] - 537s 489ms/step - loss: 5.5805 - accuracy: 0.1333
Epoch 6/20
1097/1097 [==============================] - 567s 517ms/step - loss: 5.2255 - accuracy: 0.1493
Epoch 7/20
1097/1097 [==============================] - 557s 507ms/step - loss: 4.8330 - accuracy: 0.1679
Epoch 8/20
1097/1097 [==============================] - 560s 511ms/step - loss: 4.4043 - accuracy: 0.2051
Epoch 9/20
1097/1097 [==============================] - 527s 481ms/step - loss: 3.9774 - accuracy: 0.2601
Epoch 10/20
1097/1097 [=======================

Попробуем сгенерировать текст

In [23]:
def generate_text(seed_text):
    seed_tokens = word_tokenize(seed_text)
    seed_sequence = [word_dict[word] for word in seed_tokens]

    for _ in range(50):
        input_sequence = pad_sequences([seed_sequence], maxlen=sequence_length)
        predicted_probs = model.predict(input_sequence)[0]
        predicted_word_index = np.random.choice(np.argsort(predicted_probs)[-5:][::-1])
        seed_sequence.append(predicted_word_index)
        seed_sequence = seed_sequence[-sequence_length:]

    return seed_text + ' ' + ' '.join([reverse_word_dict[index] for index in seed_sequence])

In [24]:
seed_text = 'My name is Patrick Bateman'
generated_text = generate_text(seed_text)

print(generated_text)

1/1 [==============================] - 0s 28ms/step
My name is Patrick Bateman is the right commercial Evelyn has to talk Evelyn screams up Evelyn says Shes not paying some eerie muscles dont go back or the waiter begins Uh yeah Bateman Helga asks her confused cuts me to inspect a straight at a passing book But my god Evelyn says No Im
